# LSTM
[資料來源](https://www.finlab.tw/%E7%94%A8%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92%E5%B9%AB%E4%BD%A0%E8%A7%A3%E6%9E%90K%E7%B7%9A%E5%9C%96%EF%BC%81/)

In [12]:
import pandas as pd

foxconndf= pd.read_csv('./daily_stock_3535_2.csv')

In [19]:
from sklearn import preprocessing

def normalize(df):
    newdf= df.copy()
    min_max_scaler = preprocessing.MinMaxScaler()

    newdf['ts'] = df.ts.values.reshape(-1,1)
    newdf['high'] = min_max_scaler.fit_transform(df.high.values.reshape(-1,1))
    newdf['low'] = min_max_scaler.fit_transform(df.low.values.reshape(-1,1))
    newdf['open'] = min_max_scaler.fit_transform(df.open.values.reshape(-1,1))
    newdf['close'] = min_max_scaler.fit_transform(df.close.values.reshape(-1,1))
    newdf['change'] = min_max_scaler.fit_transform(df.change.values.reshape(-1,1))
    newdf['transaction'] = df.transaction.values.reshape(-1,1)
    newdf['next_open'] = min_max_scaler.fit_transform(df.next_open.values.reshape(-1,1))
    return newdf

foxconndf_norm= normalize(foxconndf)

In [20]:
foxconndf_norm.head()

,ts,high,low,open,close,change,transaction,next_open
0,1262563200,0.554993,0.564347,0.574442,0.564508,0.510791,1057,0.563531
1,1262649600,0.550169,0.544316,0.563531,0.533133,0.338129,613,0.528371
2,1262736000,0.518813,0.545568,0.528371,0.536898,0.539568,311,0.558681
3,1262822400,0.554993,0.576865,0.558681,0.584588,0.791367,2342,0.592629
4,1262908800,0.569465,0.564347,0.592629,0.569528,0.431655,1655,0.575655


In [ ]:
import numpy as np

def data_helper(df, time_frame):
    
    # 資料維度: 開盤價、收盤價、最高價、最低價、成交量, 5維
    number_features = len(df.columns)

    # 將dataframe 轉成 numpy array
    datavalue = df.as_matrix()

    result = []
    # 若想要觀察的 time_frame 為20天, 需要多加一天做為驗證答案
    for index in range( len(datavalue) - (time_frame+1) ): # 從 datavalue 的第0個跑到倒數第 time_frame+1 個
        result.append(datavalue[index: index + (time_frame+1) ]) # 逐筆取出 time_frame+1 個K棒數值做為一筆 instance
    
    result = np.array(result)
    number_train = round(0.9 * result.shape[0]) # 取 result 的前90% instance做為訓練資料
    
    x_train = result[:int(number_train), :-1] # 訓練資料中, 只取每一個 time_frame 中除了最後一筆的所有資料做為feature
    y_train = result[:int(number_train), -1][:,-1] # 訓練資料中, 取每一個 time_frame 中最後一筆資料的最後一個數值(收盤價)做為答案
    
    # 測試資料
    x_test = result[int(number_train):, :-1]
    y_test = result[int(number_train):, -1][:,-1]
    
    # 將資料組成變好看一點
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], number_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], number_features))  

    return [x_train, y_train, x_test, y_test]

# 以20天為一區間進行股價預測
X_train, y_train, X_test, y_test = data_helper(foxconndf_norm, 20)